## Fonction de fusion des modèles

In [1]:
#Les modèles ont déjà été renommés
#On ajoute aux réactions de type EX_xxx le metabolite xxx_pool
#On ajoute la réaction xxx_pool <-> que l'on nomme "pool_xxx"
import cobra.test
from cobra import *

In [2]:
def FinderOfEX(model):
    '''
        Retrun a dictList of cobra reactions starting with EX
    '''
    res=[]
    for x in model.reactions:
        if type(x.id)==type("string") and x.id.startswith("EX"): #comparaison des types pour sauter les reactions.. 
                                                                #..sans noms : x.name de type NoneType.
            res.append(x.id)
    return res

In [53]:
def fusion(modelList):
    
    '''
    --Parameters--

    modelList est une liste de modèles. En pratique de longueur 2 ou 3
    
    --Return--
    Return the fusion of the models 
    '''
    
    from cobra import Model, Reaction, Metabolite
    #Initialisation
    NewModel = cobra.Model("Fusion des modèles !")
    
    #Ajout de toutes les réactions dans le nouveau modèle 
    
    for model in modelList:
        NewModel.add_reactions(model.reactions)
    
    
    #Modification des réactions EX comme décrit:
    EX_Reactions=[]
    for model in modelList:
        EX_Reactions+=FinderOfEX(model)
    
    #help(type(NewModel))
    for idEX in EX_Reactions:
        rootReac = NewModel.reactions.get_by_id(idEX)#Reaction avec une seule métabolite que l'on récupère:
        #print(NewModel.reactions)
        for metab in rootReac.metabolites:#Un seul métabolite #utilisation boucle car difficultés à récupérer autrement
            rootMetab = NewModel.metabolites.get_by_id(metab.id)
        poolMetab = Metabolite(
            'pool_'+rootMetab.id,
            formula=rootMetab.formula,
            compartment='pool',
            name='Version pool de '+rootMetab.name
                )
        rootReac.add_metabolites({ ##
            poolMetab : -rootMetab.charge #charge = coefficient stoechiometrique de root dans la reaction i.e. -1 normalement
        })
        #Ajout des réaction xxx_pool <-> que l'on identifient par "pool_xxx"
        poolReac = Reaction('pool_'+rootMetab.id)
        poolReac.name='Reaction : '+rootMetab.id+'_pool <->'
        poolReac.subsystem = 'pool'
        #Bounds will be set by the diet.
        poolReac.add_metabolites({
            poolMetab : -1.0
        })
        NewModel.add_reactions([poolReac])
    ##Changement de l'objectif ?
    #XXX
    return NewModel

In [62]:
fusion(modelList).objective

## TESTS

In [5]:
mini=cobra.test.create_test_model("mini")

In [6]:
# Best practise: SBML compliant IDs
modeltest = Model('example_model')

reaction = Reaction('REACTION_INSEREE')
reaction.name = 'reaction que jai insérée '
reaction.subsystem = 'FAUDRA FAIRE GAFFE AU SUBSYSTEM'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [7]:
TEST = Metabolite(
    'TEST',
    formula='FORMULETEST',
    name='NAMETEST',
    compartment='test')
reaction.add_metabolites({
    TEST : -1.0 
})


In [8]:
modeltest.add_reactions([reaction])

In [9]:
modelList=[modeltest,mini]

In [16]:
fusion(modelList)
#mini.reactions.get_by_id('EX_h_e')

KeyError: 1

L'ajout de réaction est OK !

In [151]:
FinderOfEX(mini)

['EX_glc__D_e', 'EX_h_e', 'EX_lac__D_e']

In [25]:
help(type(mini.metabolites))

Help on class DictList in module cobra.core.dictlist:

class DictList(builtins.list)
 |  A combined dict and list
 |  
 |  This object behaves like a list, but has the O(1) speed
 |  benefits of a dict when looking up elements by their id.
 |  
 |  Parameters
 |  ----------
 |  *args : iterable
 |      iterable as single argument to create new DictList from
 |  
 |  Method resolution order:
 |      DictList
 |      builtins.list
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      x.__add__(y) <==> x + y
 |      
 |      Parameters
 |      ----------
 |      other : iterable
 |          other must contain only unique id's which do not intersect
 |          with self
 |  
 |  __contains__(self, object)
 |      DictList.__contains__(object) <==> object in DictList
 |      
 |      object: str or :class:`~cobra.core.Object.Object`
 |  
 |  __copy__(self)
 |  
 |  __delitem__(self, index)
 |      Delete self[key].
 |  
 |  __delslice__(self, i, j)
 |

In [115]:
help(type(modeltest.metabolites.__getitem__(0)))

Help on class Metabolite in module cobra.core.metabolite:

class Metabolite(cobra.core.species.Species)
 |  Metabolite is a class for holding information regarding
 |  a metabolite in a cobra.Reaction object.
 |  
 |  Parameters
 |  ----------
 |  id : str
 |      the identifier to associate with the metabolite
 |  formula : str
 |      Chemical formula (e.g. H2O)
 |  name : str
 |      A human readable name.
 |  charge : float
 |     The charge number of the metabolite
 |  compartment: str or None
 |     Compartment of the metabolite.
 |  
 |  Method resolution order:
 |      Metabolite
 |      cobra.core.species.Species
 |      cobra.core.object.Object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, id=None, formula=None, name='', charge=None, compartment=None)
 |      A simple object with an identifier
 |      
 |      Parameters
 |      ----------
 |      id: None or a string
 |          the identifier to associate with the object
 |  
 |  remove_from